<a href="https://colab.research.google.com/github/cantaruttim/adai_college/blob/main/Modelagem_College.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import os
import re
from datetime import datetime, timedelta

# Modelagem das Bases

## Gera a Base dos Cursos

In [25]:
def gera_base_cursos():
  cursos = {
      # Incluir o curso no formulário de inscrição.
      # Acertar o nome dos cursos para podermos bater com a base de alunos
      "cd_curso": ["1", "2"],
      "nome_curso": ["College Intensivo", "College Intensivo Master"]
  }

  cursos = pd.DataFrame(cursos)
  return cursos

cursos = gera_base_cursos()
cursos

,cd_curso,nome_curso
0,1,College Intensivo
1,2,College Intensivo Master


## Gera a Base de alunos

In [46]:
def formatar_whatsapp(numero):
    if pd.isnull(numero):
        return numero
    numeros = re.sub(r"\D", "", numero)

    # Verifica se tem 11 dígitos (formato brasileiro com DDD + 9 dígitos)
    if len(numeros) == 11:
        ddd = numeros[:2]
        parte1 = numeros[2:7]
        parte2 = numeros[7:]
        return f"({ddd}) {parte1}-{parte2}"
    else:
        # Retorna como está se não tiver o padrão esperado
        return numero

def gerar_matricula(turma, telefone):
    turma_str = str(turma)
    primeiros_4 = turma_str[:4]
    ultimos_2 = turma_str[-2:]

    telefone_numeros = re.sub(r'\D', '', str(telefone))
    ultimos_4_telefone = telefone_numeros[-4:] if len(telefone_numeros) >= 4 else telefone_numeros

    resultado = primeiros_4 + ultimos_2 + ultimos_4_telefone

    return resultado


def gera_base_alunos(df):
    arquivo = "alunos_pb"

    df["Nome"] = df["Nome"].str.strip()
    df["Email"] = df["Email"].str.strip()

    df["Whatssap"] = df["Whatssap"].apply(formatar_whatsapp)
    df["Whatssap"] = df["Whatssap"].str.strip()

    # Aplicando a função a uma nova coluna
    df['Matricula'] = df.apply(
        lambda row:
                gerar_matricula(
                  row['Turma'],
                  row['Whatssap']
                ),
            axis=1
    )

    df["Matricula"] = df["Matricula"].astype(str)
    df["Matricula"] = pd.to_numeric(df["Matricula"], errors='coerce').fillna(0).astype(int)

    # ajustar aqui depois para pegar o curso correto
    df["curso"] = 1
    df["curso"] = df["curso"].astype(str)

    qtd_alunos_turma = (
        df.groupby("Turma")
          .size()
          .reset_index(name="qtdAlunosTurma")
    )

    df = df.merge(qtd_alunos_turma, on="Turma", how="left")

    df["qtdAlunosTurma"] = df["qtdAlunosTurma"].astype(int)

    df.to_excel(f"{arquivo}.xlsx", index=False)

df = pd.read_csv("/content/alunos.csv", sep=";")
alunos = gera_base_alunos(df)
## NA BASE DE ALUNOS PRECISAMOS INCLUIR QUAL CURSO O ALUNO IRÁ FAZER

None

In [58]:
alunos = pd.read_excel("/content/alunos_pb.xlsx")

## Gera a base Módulo

In [57]:
meses_pt_en = {
    "jan": "Jan", "fev": "Feb", "mar": "Mar", "abr": "Apr", "mai": "May", "jun": "Jun", "jul": "Jul", "ago": "Aug", "set": "Sep", "out": "Oct", "nov": "Nov", "dez": "Dec"
}

def data_valida(data_str, formato="%Y-%m-%d"):
    try:
        data = datetime.strptime(data_str, formato)
    except ValueError:
        return False  # Data inválida ou formato errado

    hoje = datetime.today()
    limite = data + timedelta(days=7)
    return hoje <= limite

def traduzir_mes(data_str):
    if not data_str:
        return None
    for pt, en in meses_pt_en.items():
        data_str = data_str.lower().replace(f"/{pt}", f"/{en}")
    return data_str

def valida_dia_semana_e_aula(df):
# Segunda-feira é 0 e Domingo é 6
  df['dia_util'] = df['data'].dt.dayofweek.apply(lambda x: 'Útil' if x < 5 else 'Não útil')

  df['tipo_atividade'] = df['atividade'].apply(lambda x: 'Aula' if isinstance(x, str) and x.startswith('Aula') else 'Sem Aula')

  return df


def retorna_modulo(modulo):
  modulo["status_data"] = modulo["DATA"].apply(lambda x: data_valida(x))

  modulo["DATA"] = modulo["DATA"].fillna("").str.strip()
  modulo["DATA"] = modulo["DATA"].apply(
      lambda x: f"{x}/2025" if x else None
  )

  modulo["DATA"] = modulo["DATA"].apply(traduzir_mes)
  modulo["DATA"] = pd.to_datetime(modulo["DATA"], format="%d/%b/%Y", errors="coerce")

  # Extrai o ano
  modulo["Ano"] = modulo["DATA"].dt.year

  modulo = modulo[["DATA", "Ano", "DIA DA SEMANA", "ATIVIDADE", "MÓDULO", "Semestre", "status_data"]]


  # Cria a coluna 'ANOMES' no formato YYYYMM
  modulo['ANOMES'] = modulo['DATA'].dt.strftime('%Y%m')

  # Agrupa por 'ANOMES' e 'módulo' e conta a quantidade de aulas
  # Corrigindo o nome da coluna de 'módulo' para 'MÓDULO'
  aulas_por_anomes_modulo = (
      modulo
        .groupby(['ANOMES', 'MÓDULO'])
        .size()
        .reset_index(name='quantidade_aulas')
  )


  modulo = modulo.merge(
      aulas_por_anomes_modulo,
      on=['ANOMES', 'MÓDULO'],
      how="left"
  )

  # Corrigindo a lista de nomes de colunas para incluir 'ANOMES'
  modulo.columns = ["data",
                    "ano",
                    "dia_semana",
                    "atividade",
                    "módulo",
                    "semestre",
                    "status_data",
                    "ANOMES",
                    "quantidade_aulas"]

  return modulo

modulo = valida_dia_semana_e_aula(
    retorna_modulo(
        pd.read_csv("/content/Módulos.csv", sep=";")
    )
)

# Agrupar por tipo_atividade e contar a quantidade
contagem_tipo_atividade = (
    modulo.groupby(
        ['módulo', 'tipo_atividade', 'semestre']
    )
    .size()
    .reset_index(name='quantidade')
)

modulo = modulo.merge(
    contagem_tipo_atividade,
    on=['módulo', 'tipo_atividade', 'semestre'],
    how="left"
)


## Gerar a Base de Atividades

In [49]:
def valida_semestre(df):
    # Converter a coluna 'Carimbo de data/hora' para datetime, se ainda não estiver
    df['Carimbo de data/hora'] = pd.to_datetime(df['Carimbo de data/hora'])

    # Extrair Ano e Mês
    df['Ano'] = df['Carimbo de data/hora'].dt.year
    df['Mes'] = df['Carimbo de data/hora'].dt.month

    # Criar a coluna ANOMES
    df['ANOMES'] = df['Carimbo de data/hora'].dt.strftime('%Y%m')

    # Determinar o Semestre
    df['Semestre'] = df['Mes'].apply(lambda x: 'Primeiro' if 1 <= x <= 6 else 'Segundo')

    return df

def valida_atividade(df):
    contagem_atividades = (
        df.groupby(['Endereço de e-mail', 'módulo', 'Semestre']) # Corrected 'módulo' to 'Módulo'
        .size()
        .reset_index(name='atividades')
    )

    df = df.merge(
        contagem_atividades,
        on=['Endereço de e-mail', 'módulo', 'Semestre'], # Corrected 'módulo' to 'Módulo'
        how="left"
    )

    return df

atv = valida_atividade(
        valida_semestre(
            pd.read_excel(
                '/content/Atividade - Módulo 5_ Livros Históricos (respostas).xlsx', sheet_name='Respostas ao formulário 1'
            )
        )
    )

atv = atv[
    ["Carimbo de data/hora",
     "Endereço de e-mail",
     "Nome completo",
     "módulo",
     "ANOMES",
     "Semestre",
     "atividades"
    ]
]

# Notas Finais

## Calcula nota da prova

In [ ]:
## LÊ O ARQUIVO DE PROVAS E CALCULA A QUANTIDADE DE RESPOSTAS CERTAS

## PARA O MASTER, MAPEAR COMO PODEMOS FAZER COM A NOTA AVALIATIVA, SENDO QUE PODE TER MAIS PROVAS PRÁTICAS.

## Calcula atividades

In [59]:
def retorna_quantidade_atividades(modulo, atv):
  modulo = modulo.rename(columns={'semestre': 'Semestre'})

  modulo["quantidade_aulas"] = modulo["quantidade_aulas"].astype(int)
  merged_df = atv.merge(
      modulo,
      on=["módulo", "Semestre"],
      how="left"
  )

  # Ajustar o condicional para determinar o status
  merged_df["status"] = (
      merged_df.apply(
        lambda row: "aprovado"
          if row["atividades"] >= row["quantidade_aulas"]
            else "não aprovado",
        axis=1
    )
  )

  return merged_df

merged_df = retorna_quantidade_atividades(modulo, atv)